## Script Description:

This notebook is to setup the basic infrastructure of the enviornment, including two machines for TCP transition and QUIC transition.

They will be named "TCP server", "TCP client", "QUIC server", "QUIC client"

We would like to test them into 3 different network connection conditions:

1. no package loss
2. with low package loss
3. with 50% package loss

One package we are going to test on QUIC is aioquic, the most popular and well-known package using Python to implemented.

In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

try:
    fablib = fablib_manager()
    fablib.show_config()
except Exception as e:
    print(f"Exception: {e}")

Orchestrator,orchestrator.fabric-testbed.net
Credential Manager,cm.fabric-testbed.net
Core API,uis.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,2aaaea18-5cf9-497a-ade0-b4f51112a34d
Bastion Host,bastion.fabric-testbed.net
Bastion Username,zhang303_0000071337
Bastion Private Key File,/home/fabric/work/fabric_config/fabric-bastion-key
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Sites to avoid,


In [24]:
slice_name = "QUIC_mini_project"

# remove the slice if it is already exist #
try:
    slice = fablib.get_slice(name=slice_name)
    slice.delete()
except:
    print("network was not exist")

# create slice
slice = fablib.new_slice(slice_name)

# listing the current slice
slice.list_nodes()

In [3]:
# init the QUIC nodes
node_QUIC_server = slice.add_node(name="QUIC_server",
                      site="SALT",
                      cores=2,
                      ram=4,
                      disk=10,
                      image="default_ubuntu_20"
)
node_QUIC_client = slice.add_node(name="QUIC_client",
                      site="SALT",
                      cores=2,
                      ram=4,
                      disk=10,
                      image="default_ubuntu_20"
)

# init the TCP nodes
node_TCP_server = slice.add_node(name="TCP_server",
                      site="SALT",
                      cores=2,
                      ram=4,
                      disk=10,
                      image="default_ubuntu_20"
)
node_TCP_client = slice.add_node(name="TCP_client",
                      site="SALT",
                      cores=2,
                      ram=4,
                      disk=10,
                      image="default_ubuntu_20"
)

In [4]:
# listing the current slice
slice.list_nodes()

ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
None,QUIC_client,None,None,None,default_ubuntu_20,qcow2,None,SALT,ubuntu,,None,,,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
None,QUIC_server,None,None,None,default_ubuntu_20,qcow2,None,SALT,ubuntu,,None,,,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
None,TCP_client,None,None,None,default_ubuntu_20,qcow2,None,SALT,ubuntu,,None,,,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
None,TCP_server,None,None,None,default_ubuntu_20,qcow2,None,SALT,ubuntu,,None,,,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
None,QUIC_client,None,None,None,default_ubuntu_20,qcow2,None,SALT,ubuntu,,None,,,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
None,QUIC_server,None,None,None,default_ubuntu_20,qcow2,None,SALT,ubuntu,,None,,,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
None,TCP_client,None,None,None,default_ubuntu_20,qcow2,None,SALT,ubuntu,,None,,,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
None,TCP_server,None,None,None,default_ubuntu_20,qcow2,None,SALT,ubuntu,,None,,,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


In [5]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
import ipaddress

# getting the random site 
site = fablib.get_random_site()
print(f"Site: {site}")

# define the net
network_name='QUIC_mini_project'

net_test = slice.add_l2network(name=network_name, 
                           subnet=IPv4Network("192.168.1.0/24")
                          )

# create iface for all 4 nodes
list_nodes = [node_QUIC_server, node_QUIC_client, node_TCP_server, node_TCP_client]
for cnode in list_nodes:
    name_cnode = cnode.get_name()
    ciface = cnode.add_component(model="NIC_Basic", 
                                         name="nic_" + name_cnode
                                        ).get_interfaces()[0]
    ciface.set_mode("auto")
    net_test.add_interface(ciface)
    

Site: MAX


In [6]:
# Save the topology requeest
slice.save("week1.graphml")

#Submit Slice Request
slice.submit();

# changed vm size since following error
# 4/1 update: encountered an error about cannot go multi-site
# "Your project is lacking VM.NoLimitCPU or VM.NoLimit tag to provision VM with more than 2 cores."
# "Your project is lacking VM.NoLimitDisk or VM.NoLimit tag to provision VM with disk over 10GB."


Retry: 10, Time: 316 sec


ID,85b34ed9-4403-4d35-96dd-8a6f6afd8a66
Name,QUIC_mini_project
Lease Expiration (UTC),2024-04-26 21:49:39 +0000
Lease Start (UTC),2024-04-25 21:49:41 +0000
Project ID,2aaaea18-5cf9-497a-ade0-b4f51112a34d
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
89ee3491-9a6a-4808-8dbc-6a0155e9bc04,QUIC_client,2,4,10,default_ubuntu_20,qcow2,salt-w2.fabric-testbed.net,SALT,ubuntu,2001:400:a100:3010:f816:3eff:fe1f:804,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3010:f816:3eff:fe1f:804,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
ee2fbb89-5a43-4753-9863-7d224915b8fe,QUIC_server,2,4,10,default_ubuntu_20,qcow2,salt-w2.fabric-testbed.net,SALT,ubuntu,2001:400:a100:3010:f816:3eff:fe62:1531,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3010:f816:3eff:fe62:1531,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
47673eaa-83e8-4122-8d83-60ad94e11259,TCP_client,2,4,10,default_ubuntu_20,qcow2,salt-w2.fabric-testbed.net,SALT,ubuntu,2001:400:a100:3010:f816:3eff:fe9e:842b,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3010:f816:3eff:fe9e:842b,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
e5ba9d45-a1aa-4b84-81c0-a58a72d4b236,TCP_server,2,4,10,default_ubuntu_20,qcow2,salt-w2.fabric-testbed.net,SALT,ubuntu,2001:400:a100:3010:f816:3eff:fea1:afc5,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3010:f816:3eff:fea1:afc5,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
77949c79-9a7d-4589-ab4b-e4d7d6ed8045,QUIC_mini_project,L2,L2Bridge,SALT,192.168.1.0/24,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node
QUIC_server-nic_QUIC_server-p1,p1,QUIC_server,QUIC_mini_project,100,auto,,16:3B:9C:3F:A4:A6,enp7s0,enp7s0,192.168.1.3,4
QUIC_client-nic_QUIC_client-p1,p1,QUIC_client,QUIC_mini_project,100,auto,,16:9D:CD:05:FC:F9,enp7s0,enp7s0,192.168.1.1,4
TCP_server-nic_TCP_server-p1,p1,TCP_server,QUIC_mini_project,100,auto,,1E:E0:77:40:06:28,enp7s0,enp7s0,192.168.1.4,4
TCP_client-nic_TCP_client-p1,p1,TCP_client,QUIC_mini_project,100,auto,,1E:50:ED:95:BA:A5,enp7s0,enp7s0,192.168.1.2,4



Time to print interfaces 316 seconds


In [7]:
slice_name = "QUIC_mini_project"

# remove the slice if it is already exist #
try:
    slice = fablib.get_slice(name=slice_name)
    print(slice)
except:
    print("network was not exist")

-----------  ------------------------------------
Slice Name   QUIC_mini_project
Slice ID     85b34ed9-4403-4d35-96dd-8a6f6afd8a66
Slice State  StableOK
Lease End    2024-04-26 21:49:39 +0000
-----------  ------------------------------------


In [8]:
node_names = ["QUIC_server", "QUIC_client", "TCP_server", "TCP_client"]

for n in node_names:
    # getting the node
    cur_node = slice.get_node(name=n)
    if("QUIC" in n):
        # executing the package installation
        cur_node.execute("sudo apt-get update")
        cur_node.execute("sudo apt install -y libssl-dev python3-dev")
        cur_node.execute("sudo apt install -y python3-pip")
        # will need HTTP/3 on aioquic example
        cur_node.execute("git clone https://github.com/aiortc/aioquic.git")
        cur_node.execute("pip3 install aioquic dnslib jinja2 starlette wsproto")
        cur_node.execute("git clone https://github.com/steinwurf/tcp-test.git")
    else:
        cur_node.execute("sudo apt-get update")
        cur_node.execute("sudo apt install -y libssl-dev python3-dev")
        cur_node.execute("sudo apt install -y python3-pip")
        #cur_node.execute("git clone https://github.com/steinwurf/tcp-test.git")
        #cur_node.execute("sudo add-apt-repository ppa:deadsnakes/ppa")
        #cur_node.execute("sudo apt-get update")
        cur_node.execute("sudo apt install -y python3.12")
        #cur_node.execute("git clone https://github.com/ccie18643/PyTCP.git")
        #cur_node.execute("curl -sS https://bootstrap.pypa.io/get-pip.py | python3.12")
        #cur_node.execute("/home/ubuntu/.local/bin/pip install PyTCP")
        #cur_node.execute("sudo apt install python3.12-venv")
        #cur_node.execute("sudo apt-get install -y bridge-utils")
        #cur_node.execute("sudo apt-get install -y build-essential")
    

Hit:1 http://nova.clouds.archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:4 http://nova.clouds.archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:5 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe amd64 Packages [8628 kB]
Get:6 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2892 kB]
Get:7 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe Translation-en [5124 kB]
Get:8 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe amd64 c-n-f Metadata [265 kB]
Get:9 http://nova.clouds.archive.ubuntu.com/ubuntu focal/multiverse amd64 Packages [144 kB]
Get:10 http://nova.clouds.archive.ubuntu.com/ubuntu focal/multiverse Translation-en [104 kB]
Get:11 http://nova.clouds.archive.ubuntu.com/ubuntu focal/multiverse amd64 c-n-f Metadata [9136 B]
Get:12 http://nova.clouds.archive.ubuntu.c

In [23]:
node_names = ["QUIC_server", "QUIC_client", "TCP_server", "TCP_client"]

for n in node_names:
    # starting the file upload on "upload" folder #
    cur_node = slice.get_node(name=n)
    cur_node.execute("mkdir ~/uploads")
    cur_node.upload_directory(local_directory_path="./scripts/", remote_directory_path="~/uploads/.")

 mkdir: cannot create directory ‘/home/ubuntu/uploads’: File exists
  mkdir: cannot create directory ‘/home/ubuntu/uploads’: File exists
  mkdir: cannot create directory ‘/home/ubuntu/uploads’: File exists
  mkdir: cannot create directory ‘/home/ubuntu/uploads’: File exists
 

In [10]:
fablib.get_slices()

# deleting the whole slice (remove if it is for network init)
# slice.delete()

In [11]:
# this will create the config files for me.
fablib.create_ssh_config(overwrite=True)

In [20]:
node_names = ["QUIC_server", "QUIC_client", "TCP_server", "TCP_client"]

for n in node_names:
    # getting the node
    cur_node = slice.get_node(name=n)
    cur_addr = cur_node.get_interface(network_name=network_name).get_ip_addr()
    print([n, cur_addr])

['QUIC_server', IPv4Address('192.168.1.3')]
['QUIC_client', IPv4Address('192.168.1.1')]
['TCP_server', IPv4Address('192.168.1.4')]
['TCP_client', IPv4Address('192.168.1.2')]


In [13]:
# python3 quic_server.py --host 192.168.1.1 -c cert.pem -k pkey.pem -p 9999
# python3 quic_client.py --host 192.168.1.1 -p 9999 1

# on gate: sudo vim /etc/hosts
# on gate: enter "192.168.1.3 yifang.com"
# server: python3 examples/http3_server.py --certificate ../uploads/cert.pem --private-key ../uploads/pkey.pem --host yifang.com --password Abcd1234
# client: python3 

In [14]:
# finding the local ethernet:
# https://unix.stackexchange.com/questions/14961/how-to-find-out-which-interface-am-i-using-for-connecting-to-the-internet
# net-tools and "route" command
"""
Kernel IP routing table
Destination     Gateway         Genmask         Flags Metric Ref    Use Iface
10.30.6.0       0.0.0.0         255.255.254.0   U     0      0        0 enp3s0
169.254.169.254 10.30.6.11      255.255.255.255 UGH   100    0        0 enp3s0
192.168.1.0     0.0.0.0         255.255.255.0   U     0      0        0 enp7s0
"""
# changing the network reliability
# ip link show
# sudo apt install net-tools
# route
# sudo tc qdisc del dev enp7s0 root 2>/dev/null
# sudo tc qdisc add dev enp7s0 root handle 1:0 netem
# sudo tc qdisc add dev enp7s0 root handle 1:0 netem loss 5%

'\nKernel IP routing table\nDestination     Gateway         Genmask         Flags Metric Ref    Use Iface\n10.30.6.0       0.0.0.0         255.255.254.0   U     0      0        0 enp3s0\n169.254.169.254 10.30.6.11      255.255.255.255 UGH   100    0        0 enp3s0\n192.168.1.0     0.0.0.0         255.255.255.0   U     0      0        0 enp7s0\n'